# Finding Building Owners

Research question: who owns the largest buildings in Seattle? How much pollution are those buildings responsible for?

Research challenge: We have a dataset of large buildings (>20k sq ft) and their carbon footprint from the [City of Seattle's Office of Sustainability and the Environment](https://www.seattle.gov/environment/climate-change/buildings-and-energy/energy-benchmarking). This means that we can group those buildings by owner and determine the carbon footprint of each owner. However,the City of Seattle does not provide us with the owners of buildings in the carbon benchmarking dataset. Without this data, we can't determine which companies have the largest environmental impact on Seattle. Moreover, the companies listed as building owners in the King County property database are often shell companies. There is no database where you can look up the parent company for a shell company. There is also no reliable way to find a parent company's shell companies (though this assumes you're looking for buildings owned by one particular company).

For the purpose of the report, we focused our search on buildings in the downtown neighborhood that are over 100 sq. ft. We decided this for political reasons but also because downtown is where a significant portion of the largest buildings in Seattle are located, and larger buildings seemed to on average have higher carbon emissions intensities. The benefit of extensible code is that we can change our minds at any time and select a different subset of the data (perhaps we want to consider a particular council district) and have results within a few minutes. 


**Vocabulary used in this tutorial**

- [Corporations and Charities Filings System](https://www.sos.wa.gov/corporations-charities): a publicly-available government database listing all companies and non-profits in Washington State.
- [eRealProperty](https://blue.kingcounty.com/Assessor/eRealProperty/default.aspx): a publicly-available government database that lists the owners of all real estate parcels in King County (where Seattle is located).
- Parent company: a company that owns subsidiary companies.
- Principal: a person who is an officer or director of a company. A company must list at least one principal to be incorporated.
- Shell company: a company owned by a parent company that disguises its relationship with that parent company. Creating a different shell company for every building owned by a parent company is a common tactic to hide the extent of a company's real estate holdings.
- Tax parcel owner: the result of looking up an address in eRealProperty. Also referred to as the building owner in this tutorial. 

Finding a building's owner is then a three-step process:

0. Get a list of buildings that you want to determine ownership for. In our use case, we are looking at buildings listed in Seattle's Office of Sustainability's (OSE) building emissions reports. 
1. Find the parcel owner for the lot the building is on, using the eRealProperty website. 
2. Given the parcel owner from Step 1, search the Washington State Corporations and Charities Filings database for that company and collect the company's principals.
3. Search the company principals on Google and see what company they work for based on Linkedin, business journal articles, etc. Whichever company they all work for (or worked for most recently) is the parent company that owns the building.

A note on methodology: parent companies re-use the same people as principals for their shell companies again and again. The principals also tend to stay with a company for a long time. This is why searching for the current or most recent employer for each of the principals will get you the parent company in most cases. Where this doesn't work, some digging and a bit of common sense (when was the company incorporated? Where were these people working at that time, according to Linkedin?) will get you the rest of the way.

Steps 1 and 2 are done programmatically. Step 3 is done by hand. The steps are outlined below.

In [22]:
# Be sure to run `pip install -r requirements.txt` in the root directory
from utils import corp_owners
from utils import geo
from utils import parcel_owners

In [23]:
import pandas as pd
import numpy as np
import requests
import json
import os
import re
import geopandas as gp
import urllib.parse

# Step 0: Clean up Seattle OSE Data and Identify Buildings of Interest
Seattle's [Office of Sustainability and Environment](https://www.seattle.gov/environment) has released emissions data for buildings in the city from 2015 to 2021. Building owners over 20,000 sq.ft. are required to self-report annually to the city of Seattle. Features of the data include:
- building name
- address
- tax parcel identification number
- council district the building is located within
- buliding type
- different metrics for emissions. 
More metadata about the energy benchmarking datasets can be found [here](https://data.seattle.gov/Community/2021-Building-Energy-Benchmarking/bfsh-nrm6).

When doing initial exploratory analysis, we noticed that not all of the council district codes were correct. To rectify this we got the official council district boundaries form [Seattle GeoData Portal](https://data-seattlecitygis.opendata.arcgis.com/datasets/seattle-city-council-districts-for-council-members-serving-through-2023/explore) in the form of a geojson file [`Council_Districts.geojson`](../../../data/Council_Districts.geojson). For each address in the OSE building efficiency dataset, we convert it to a (latitude, longitude) point and confirmed with which council distirct it belonged to. To learn more, refer to `utils/corp_owners.py`.

In [6]:
# Clean the OSE dataset 
df_districts = gp.read_file('data/raw_input/Council_Districts.geojson')
df = pd.read_csv('data/raw_input/2020_Building_Energy_Benchmarking.csv')
df = gp.GeoDataFrame(df, geometry=gp.points_from_xy(df.Longitude, df.Latitude))
geo.clean_districts(df, df_districts)

# Get all the buildings in the downtown neighborhood
# Note this is a slightly different bounds than council districts
df_filtered = df.loc[df['Neighborhood'] == "DOWNTOWN"]

Building UNION HARBOR CONDOMINIUM 454/ 8807200000 doesn't have a district POINT (-122.33003 47.6401) 
	 Found district 4 for UNION HARBOR CONDOMINIUM
Building WATERWORKS OFFICE & MARINA 1494/ 4088803975 doesn't have a district POINT (-122.33895 47.63575) 
	 Found district 7 for WATERWORKS OFFICE & MARINA
Building NAUTICAL LANDING 1742/ 4088804350 doesn't have a district POINT (-122.34219 47.64306) 
	 Found district 7 for NAUTICAL LANDING
Building THE PIER AT LESCHI 3453/ 6780900000 doesn't have a district POINT (-122.28563 47.59926) 
	 Found district 3 for THE PIER AT LESCHI
Building EDUCARE 3496/ 2895800030 doesn't have a district POINT EMPTY 
Building THE LAKESHORE 3506/ 1180001715 doesn't have a district POINT EMPTY 


In [7]:
df_filtered.head()

,OSEBuildingID,TaxParcelIdentificationNumber,DataYear,BuildingType,BuildingName,Owner,CouncilDistrictCode,Neighborhood,Units,YearBuilt,...,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,Outlier,ComplianceIssue,ComplianceStatus,Comments,DefaultData,LegislationPropertyType,geometry
2,50160,659000775,2020,NonResidential,AMAZON DOPPLER BUILDING,ACORN DEVELOPMENT LLC,7.0,DOWNTOWN,NaN,2016,...,448625.0,Convention Center,59672.0,NaN,No Issue,Compliant,NaN,NaN,Office,POINT (-122.33835 47.61523)
4,50192,660001605,2020,NonResidential,MIDTOWN 21 (AMAZON),MIDTOWN21 LLC,7.0,DOWNTOWN,NaN,2016,...,110813.0,NaN,NaN,NaN,No Issue,Compliant,NaN,NaN,Office,POINT (-122.33304 47.61632)
13,50304,660002125,2020,Multifamily HR (10+),KINECTS TOWER,1823 MINOR WPT LLC +1823 MINOR MM LLC,7.0,DOWNTOWN,NaN,1970,...,117664.0,NaN,NaN,NaN,No Issue,Compliant,NaN,NaN,Multifamily Housing,POINT (-122.33151 47.61711)
31,50172,524780-0100,2020,Multifamily MR (5-9),80 MAIN APARTMENTS,NOT FOUND,7.0,DOWNTOWN,NaN,2016,...,1136.0,NaN,NaN,NaN,No Issue,Compliant,NaN,NaN,Multifamily Housing,POINT (-122.33508 47.60024)
40,50322,695000225,2020,Multifamily MR (5-9),MINNIE FLATS,101 DENNY LLC,7.0,DOWNTOWN,NaN,2016,...,NaN,NaN,NaN,NaN,No Issue,Compliant,NaN,NaN,Multifamily Housing,POINT (-122.35480 47.61860)


## Step 1: Getting Parcel Owners from eRealProperty

The first step in finding a building's owner is to find the owner of the building parcel. This is listed in [King County's eRealProperty database](https://blue.kingcounty.com/Assessor/eRealProperty/default.aspx). 

This script takes a list of buildings' Tax Parcel Identification Number and returns a CSV listing the current owners according to eRealProperty. Optionally, you can also produce a JSON file with the number, types, and square footage of the different units in the building.

To use:

1. Instantiate an instance of the `ParcelLookupHelper` class, including the file path where you want to save your results.
2. Run the `scrape_parcel_owners` method. Params:
    - `tax_parcel_id_numbers` (list): a list of the tax parcel IDs you want to look up
    - `file_name` (str): the file name to save the results
    - `get_unit_details` (bool): whether or not to create a JSON file of the types of units in each building. Defaults to `False`. 

Sample use:

```
scraper = ParcelLookupHelper('building_owners')
scraper.scrape_parcel_owners([659000030, 659000220], 'building_owners_grp_1', True)
```

Two important notes: 

- This is a web scraping script, so it is highly dependent on the HTML structure of the Property Detail pages. Test it to make sure you're getting the correct data in case the HTML structure has changed since this script was written.
- eRealProperty restricts you to 1,000 calls a day from a given IP address. You can divide your buildings into chunks of 1,000 and do this over several days or divide them between team members. Using different AWS or Google Cloud instances is possible but probably a bit rude. You could also use a VPN for similar results. Either way, you'll have to manually chunk this up into calls of 1,000 at least.


In [8]:
# NB: This example uses only ten buildings, so it can be run quickly. 
# When you run it with a lot of buildings, this will likely take a long time to run!

scraper = parcel_owners.ParcelLookupHelper(os.getcwd())
scraper.scrape_parcel_owners(df_filtered['TaxParcelIdentificationNumber'][:10], 'building_owners_downtown')

Processing row 0 of 10 parcel owners
Writing parcel owners to CSV: /Users/ihc/programming/350_seattle/350_seattle_building_ownership/building_owners_downtown.csv
Writing parcel owners to CSV: /Users/ihc/programming/350_seattle/350_seattle_building_ownership/building_owners_downtown.csv


In [11]:
building_owners = pd.read_csv("building_owners_downtown.csv", index_col=0)
building_owners.head()

,TaxParcelIdentificationNumber,Owner
0,659000775,ACORN DEVELOPMENT LLC
1,660001605,MIDTOWN21 LLC
2,660002125,1823 MINOR WPT LLC +1823 MINOR MM LLC
3,524780-0100,NOT FOUND
4,695000225,101 DENNY LLC


In [12]:
# Map tax ids to landlord name
d = pd.Series(building_owners.Owner.values, index=building_owners.TaxParcelIdentificationNumber).to_dict()
df_filtered['ParcelOwner'] = df_filtered['TaxParcelIdentificationNumber'].map(lambda row: d.get(row, ""))

/usr/local/lib/python3.11/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


## Step 2: Getting Company Principals from Washington State Corporations and Charities Filing System
The main problem we're trying to solve is that there are multiple data sources (OSE building benchmark, Washington tax parcel info, and now corporation filings), and elements in one source don't always precisely map to elements in another. In this step, we take the tax parcel owner identified in step 1 and try to figure out the corresponding business in the [Washington Corporations and Charities Filing System (CCFS)](https://www.sos.wa.gov/corporations-charities). 

Once we've identified the equivalent entry in the CCFS database, we can pull that business's principals. The principal of a company is usually someone who has a level of ownership and responsibility over the company. When a large corporation is creating lots of holding LLCs, they tend to have a select group of people act as principals across all the shell companies. 

We use the listed principals as a one datapoint of many for determining what larger corporation owns this shell company. If we find a group of people listed as principals on a bunch of companies, we can 1) usually quickly find these people's employer and 2) quickly determine that all these companies with the same principals likely are all owned by the same, larger corporation. 




### Step 2.1: Map Parcel Owner Names to Businesses in the CCF Database
The CCFS database's search engine tends to return almost any partial string match, which is overwhelming, but we can programmatically look up a business name and with some regex string matching magic, we can usually find the equivalent business in the CCFS database. For example, the tax parcel owner `ACORN DEVELOPMENT LLC` might show up in the CCFS database as `ACORN DEVELOPMENT L.L.C.`. The CCFS database doesn't have the best search engine ever, but we can automatically confirm on our end that the CCFS entry is equivalent to the tax parcel owner name. 

That being said, regex string matching can only do so much for us. Sometimes, we need a human in the loop to identify if a search result in the CCFS database really does match up with the tax parcel owner. That's why when running `lookup_helper.get_company_matches_and_export`, there are three output files: 

- `Exact_matches_i.csv`: results where search term matches a result in CCFS database.  
- `Potential_matches_i.csv`: when search term doesn't exactly match, there needs to be some human verification here to determine.  
- `Additional_matches_i.csv`: very weak matches in case potential_matches didn't yield enough results. 

Where `i` is the batch number. Looking up entries in the CCFS database can be time-consuming, so splitting up your tax parcel owner names into batches might be helpful. 

These output files will have a mapping from the tax parcel owner, which is referred to as the `SearchTerm`, to the equivalent CCFS entry, indicated with `BusinessName`. A human will need to go into the `potential_matches_i.csv` and add the column `is_match` to flag if the search term and the business name are likely to be equivalent. An example of this is that `SATO FAMILY LTD PARTNERSHIP` (the search term, ie. the tax parcel owner name) and `SATO FAMILY LIMITED PARTNERSHIP` (the result in the CCFS database) are highly likely to be the same companies. 

In [13]:
# Get a list of all the unique tax parcel owners identified in previous steps
unique_not_downtown_owners = df_filtered['ParcelOwner'].unique()
unique_not_downtown_owners = pd.DataFrame(unique_not_downtown_owners, columns=['owner_name'])
unique_not_downtown_owners = unique_not_downtown_owners[~unique_not_downtown_owners['owner_name'].isin(['NOT FOUND', 'UNDEFINED'])]
unique_not_downtown_owners.to_csv('unique_downtown_owners.csv')
owner_search_list = list(unique_not_downtown_owners['owner_name'])

In [14]:
lookup_helper = corp_owners.LookupCompaniesHelper(os.getcwd()) # Pass the directory to which you want the output cvs's saved
lookup_helper.get_company_matches_and_export(owner_search_list[:10], 1) # Processing only 1 batch of data

Saving output files to /Users/ihc/programming/350_seattle/350_seattle_building_ownership


### Step 2.2: Get all companies and their principals
Now that we've mapped tax parcel owner names to their entries in the CCFS database, we create a list of all the companies and all the principals registered to that company. For each company in our list, there will be a row in the output for each principal we found when looking up the company in the CCFS database. This means there are more likely than not multiple rows in the output that correspond to one company. This helps us group together companies in later steps to help determine which companies might all be owned by the same corporation, saving the human researchers some time. 


For potential matches, make sure to go in and manually create and label the `is_match` column. 

In [15]:
group_helper = corp_owners.GroupCompaniesHelper(os.getcwd(), "companies_and_principals.csv") # The output folder and file name to save to 

In [24]:
num_batches = 1  # Increase the range based on how many batches you processed previously

# For brevity, we will only search the exact matches found in Step 1
exact_match_df = pd.read_csv('exact_matches_1.csv')
all_matches = group_helper.get_companies_principals(exact_match_df)

### Step 2.3: Group Companies by shared principals
Now that we have all of the companies in CCFS database and all of the principals registered to that company, we can group all of the results by shared principals. This isn't absolutely vital, but can help the human researchers later on save time by not having to do duplicated work. The output data has the following features: 

- SearchTerm: Original tax parcel owner name for the building in the OSE building emissions dataset (aka Landlord)
- BusinessName: The business name in the CCFS database that we have matched to the SearchTerm
- PotentialRelatedCompany: A company that may be related to the company in the BusinessName field. If this field is the same as BusinessName, the row represents the "parent" or "hub" company that we are trying to match companies to
- UBINumber: ID number
- BusinessId: ID number
- Address: Address of the PotentialRelatedCompany
- Status: If the company is active/closed, etc.
- Principals: A comma separated, alphabetized list of the PotenitalRelatedCompany's principals 
- isMatch: Your best guess about whether or not the PotentialRelatedCompany is connected to the BusinessName
- notes: any useful notes about the company or explaining the isMatch value

With this final step complete, we can pass off the data to the human researchers who will now use all the provided info to determine what larger corporation owns these holding companies, and from there map out the true ownership of buildings in Seattle. 

In [25]:
companies_and_matches = group_helper.group_companies_by_principals(all_matches)

Saving to /Users/ihc/programming/350_seattle/350_seattle_building_ownershipcompanies_and_principals.csv
Processing row 0 of principal_match_list, results is 1


# Step 3: Finding the Principals' Employer
Automation can only take us so far, and at some point a human needs to come in and use their search-engine super powers to determine building ownership. In our experience, building name and address were useful datapoints as well as company principals. For more information see the rest of the Electrify report. 